# 1. Softmax Classification

## Import

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility
torch.manual_seed(1)

## Softmax
We can convert numbers to probabilities with softmax

$$ P(class = i) = \frac{e^i}{\sum{e^i}} $$

In [3]:
z = torch.FloatTensor([1, 2, 3])

PyTorch has a `softmax` function.
* dim: dimension에 따른 축을 normalize, 즉 dim=0인 경우 열의 합이 1, dim=1인 경우 행의 합이 1이 된다.

In [5]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)
print(hypothesis.sum())

tensor([0.0900, 0.2447, 0.6652])
tensor(1.)


## Cross Entropy Loss (Low-level)
For multi-class classification, we use the cross entropy loss.

$$ L = \frac{1}{N} \sum -y \log{(\hat{y})} $$

where $\hat{y}$ is the predicted probability and $y$ is the correct probability (from 0 to 1)

In [6]:
z = torch.rand(3, 5, requires_grad=True) # uniformly random
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


* num of classes = 5
* num of samples = 3

In [21]:
# 정답을 랜덤하게 생성
y = torch.randint(5, (3,)).long() # low는 default로 0, high는 5, output tensor size는 (3,)
print(y)

tensor([4, 2, 3])


위의 정답 값을 hypothesis와 같은 사이즈의 one hot vector로 표현

In [22]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1) # in-place 연산, dim=1, y의 사이즈를 (3,)에서 (3,1)으로 만든 후 1을 뿌림

tensor([[0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]])

In [23]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean() # cross-entropy
print(cost)

tensor(1.7849, grad_fn=<MeanBackward0>)


## Cross Entropy Loss with torch.nn.functional

In [24]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)

In [25]:
# high level
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)

In [26]:
# Low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.7849, grad_fn=<MeanBackward0>)

In [27]:
# High level - negative log likelihood loss
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.7849, grad_fn=<NllLossBackward0>)

PyTorch also has `F.cross_entropy` that combines `F.log_softmax()` and `F.nll_loss()`.

In [28]:
F.cross_entropy(z, y)

tensor(1.7849, grad_fn=<NllLossBackward0>)

## Training with Low-level Cross Entropy Loss

In [30]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]] # 8 samples, 4 input vectors(dim)
y_train = [2, 2, 2, 1, 1, 1, 0, 0] # 3 classes
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train) # discrete

In [33]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # Cost 계산
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d} / {} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0 / 1000 Cost: 1.098612
Epoch  100 / 1000 Cost: 0.901535
Epoch  200 / 1000 Cost: 0.839114
Epoch  300 / 1000 Cost: 0.807826
Epoch  400 / 1000 Cost: 0.788472
Epoch  500 / 1000 Cost: 0.774822
Epoch  600 / 1000 Cost: 0.764449
Epoch  700 / 1000 Cost: 0.756191
Epoch  800 / 1000 Cost: 0.749398
Epoch  900 / 1000 Cost: 0.743671
Epoch 1000 / 1000 Cost: 0.738749


## Training with F.cross_entropy

In [34]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # Cost 계산
    # hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    # y_one_hot = torch.zeros_like(hypothesis)
    # y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    # cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d} / {} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0 / 1000 Cost: 1.098612
Epoch  100 / 1000 Cost: 0.761050
Epoch  200 / 1000 Cost: 0.689991
Epoch  300 / 1000 Cost: 0.643229
Epoch  400 / 1000 Cost: 0.604117
Epoch  500 / 1000 Cost: 0.568255
Epoch  600 / 1000 Cost: 0.533922
Epoch  700 / 1000 Cost: 0.500291
Epoch  800 / 1000 Cost: 0.466908
Epoch  900 / 1000 Cost: 0.433507
Epoch 1000 / 1000 Cost: 0.399963


## High-level Implementation with nn.Module

In [36]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # num of output classes = 3
    
    def forward(self, x):
        return self.linear(x)

In [37]:
model = SoftmaxClassifierModel()

In [38]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # Cost 계산
    # hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    # y_one_hot = torch.zeros_like(hypothesis)
    # y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    # cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    # z = x_train.matmul(W) + b
    # cost = F.cross_entropy(z, y_train)
    
    # H(x) 계산
    prediction = model(x_train)
    
    # Cost 계산
    cost = F.cross_entropy(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d} / {} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0 / 1000 Cost: 2.894385
Epoch  100 / 1000 Cost: 0.713320
Epoch  200 / 1000 Cost: 0.631103
Epoch  300 / 1000 Cost: 0.573847
Epoch  400 / 1000 Cost: 0.523610
Epoch  500 / 1000 Cost: 0.476214
Epoch  600 / 1000 Cost: 0.430080
Epoch  700 / 1000 Cost: 0.384448
Epoch  800 / 1000 Cost: 0.338987
Epoch  900 / 1000 Cost: 0.294060
Epoch 1000 / 1000 Cost: 0.253871
